In [1]:
import pandas as pd


# Load the dataset
df = pd.read_excel('WaitData.Published.xlsx', sheet_name = 'F3')
df.head()




,Wait,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,LineCount2,LineCount3,LineCount4,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,0,372,62,6,249,6,5,6,6,4,...,4,2,0,0,0,0,3,1,1,0
1,-3,335,67,3,195,5,4,6,6,5,...,4,1,2,0,32,0,2,1,2,0
2,-25,274,68,4,150,4,5,6,4,6,...,4,1,1,1,13,16,1,1,3,0
3,36,369,73,2,179,5,6,4,5,5,...,3,1,2,1,45,16,3,1,1,0
4,-21,438,73,5,192,6,5,5,6,5,...,3,0,2,1,32,15,4,0,0,1


In [2]:
from sklearn import linear_model
X = df.copy()
Y = X.pop('Wait')

model = linear_model.LinearRegression()

model.fit(X, Y)
Ypred = model.predict(X) # use trained regression model to predict
r = Y-Ypred # compute prediction error (residual)
e = abs(r).mean() # compute model error
e

23.166288936348014

In [3]:
if True: # just in case I want to disable this part
    print('\n>Python feature selection:')
    from sklearn.feature_selection import RFE
    from itertools import compress
    for nFeatures in range(1,4):
        rfe = RFE(model, n_features_to_select=nFeatures)
        X_rfe = rfe.fit_transform(X,Y) #transforming data using RFE   
                #Fitting the data to model
        model.fit(X_rfe,Y)
        #print(rfe.support_)
        #print(rfe.ranking_)
        cols = list(compress( X.columns, rfe.support_))
        model.fit(X[cols],Y)
        e = abs(Y-model.predict(X[cols])).mean()
        print(e, cols)


>Python feature selection:
32.50541172120655 ['CardiacCount']
31.22493770698274 ['CardiacCount', 'DelayedInLine']
31.257528948689313 ['CardiacCount', 'StartTime', 'DelayedInLine']


In [4]:
# Run Python feature selection
if True: # just in case I want to disable this part
    print('\n>Python feature selection:')
    models = []
    best_features = []
    scores = []
    for name in X.columns:
        model.fit(X[[name]],Y)
        e = abs(Y-model.predict(X[[name]])).mean()
        models.append((name, e))
    best_features.append(min(models, key=lambda x: x[1])[0])
    scores.append(min(models, key=lambda x: x[1])[1])
    for _ in range(15):
        models = []
        cols = best_features.copy()
        for name in X.columns:
            cols.append(name)
            model.fit(X[cols],Y)
            e = abs(Y-model.predict(X[cols])).mean()
            models.append((name, e))
            cols.pop()
        best_features.append(min(models, key=lambda x: x[1])[0])
        scores.append(min(models, key=lambda x: x[1])[1])
    i = 1
    for name, value in zip(best_features, scores):
        print(i, name, value)
        i += 1


>Python feature selection:
1 DelayedInLine 31.22705769973352
2 LineCount0Strict 30.137965863467567
3 LineCount0 28.696490632253404
4 SumDelayWaitingInLine 28.246309585132362
5 AheadCount 28.14700684216455
6 NumCompletedToday 24.989574734774788
7 NumScheduledNextW2 24.659186407251344
8 DelayCount 24.329071006187295
9 InProgressSize 24.071936786758204
10 StartTime4 23.8465094920652
11 AvgDelayForDay 23.692619398547045
12 AvgWaitLastK1Customers 23.627914296582187
13 SumDelayInProgress 23.57915967047975
14 NumScheduledNextSlot 23.548081793881668
15 IsFirst 23.520963512553394
16 NumCustomersInLastW3 23.486208488960372


In [6]:
if True:  # просто на случай, если я захочу отключить эту часть
    print('\n>Python feature selection:')
    models = []

    # Циклы для перебора всех возможных комбинаций трех признаков
    for i in range(len(X.columns)):
        for j in range(i + 1, len(X.columns)):
            for k in range(j + 1, len(X.columns)):
                f1 = X.columns[i]
                f2 = X.columns[j]
                f3 = X.columns[k]
                
                # Обучение модели с тремя признаками
                model.fit(X[[f1, f2, f3]], Y)
                
                # Вычисление ошибки модели
                e = abs(Y - model.predict(X[[f1, f2, f3]])).mean()
                
                # Добавление результата в список моделей
                models.append((e, f1, f2, f3))
    
    # Сортировка моделей по значению ошибки
    models.sort()


>Python feature selection:


In [7]:
print(models[0])

(26.072745167091213, 'AheadCount', 'SumDelayWaitingByExamCode', 'NumCompletedToday')
